# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín



### Integrantes:

* Gonzalo Pinto Perez
* Yeremi Martin Huaman Torres

# 1. Preeliminares

Para empezar vamos a cargar las librerias que utilizaremos durante el desarrollo de la práctica

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.impute import *
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn import FunctionSampler
from imblearn.pipeline import make_pipeline
from scipy.stats import shapiro
from sklearn.compose import make_column_transformer
from sklearn.metrics import *
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from sklearn.compose import make_column_selector, make_column_transformer




# Importamos nuestro propio fichero de utilidades
import md_grupoa_practica1extra_ficheroutilidad as utils



## *1.1 Variables globales*

Fijamos la semilla que utilizaremos:

In [ ]:
seed = 27912

Fijamos el tamaño del conjunto de entrenamiento:

In [ ]:
train_size = 0.7

## *1.2 Funciones auxiliares*

Estas son las funciones auxiliares que hemos creado para esta práctica y que hemos añadido a nuestro utils:

In [ ]:
# =============================================================================
# Funciones creadas por nosotros
# =============================================================================
    
#Funcion para realizar gráficos de caja
def plot_boxplot(data):
    
    var = data.columns
    data = widgets.fixed(data)

    widgets.interact(_plot_boxplot, data=data, var=var)

#Función auxiliar para gráficos de caja    
def _plot_boxplot(data, var):
    return data[var].iplot(kind="box")

#Función para calcular los outliers de las variables pasadas por parámetros para un conjunto de datos pasado por paámetros también 
def calc_outliers(data, variables):
    min=99999
    max=0
    total=0
    total_variables=len(variables)
    for var in variables:
        outlier = []
        aux= data[var]
        q1= np.percentile(aux,25)
        q3= np.percentile(data[var],75)
        IQR= q3-q1
        UF= q3+(1.5*IQR)
        LF= q1-(1.5*IQR)
        for s in data[var].values:
            if(s>UF or s<LF):
                outlier.append(s)
        if(len(outlier)<min):
            min= len(outlier)
            minvar= var
        if(len(outlier)>max):
            max=len(outlier)
            maxvar=var
        print(f"El número de outliers de la variable {var} es: {len(outlier)}")
        print(f"Siendo los outliers los siguientes: {outlier}")
        print("")
        total=total+len(outlier)
    print("----Conclusiones----")
    print(f"El total de outliers que tenemos en el conjunto de datos es de: {total}")
    print(f"Con una media de  {total/len(variables)} outliers por variable")
    print(f"El mayor número de outliers lo encontramos en la variable {maxvar} con {max} outliers")
    print(f"El menor número de outliers lo encontramos en la variable {minvar} con {min} outliers")
    print("--------------------")



#Función para calcular el porcentaje de ceros en variables que no deberían de tener ceros
def ZeroCount(Data, param):
    for s in param:
        aux=Data[s]
        zeros=aux.astype(bool).sum(axis=0)
        totalval=np.product(aux.shape)
        result= (1-(zeros/totalval)) * 100    
        #print(result)
        print(f"El porcentaje de ceros en la variable {s} es del {result:.2f}% ")
        
#Función generar un diagrama de barras para analizar la potencia discriminativa de los valores perdidos de una variable con respecto a la variable clase
def _ValPerdidos_VarClase(data,param,clase):
    aux = data.copy()
    ValPerdidos = []
    
    #Comprobamos todos los casos en los que la variable dada tiene valores perdidos
    for i in aux.index:
        if(aux.loc[i,param]==0):
            ValPerdidos.append('ConValoresPerdidos')
        else:
            ValPerdidos.append('SinValoresPerdidos')
            
    #Añadimos un nuevo atributo indicando los casos en los que hay valores perdidos
    aux.loc[:,'valperdidos']= ValPerdidos
    
    #Generación del diagrama de barras apilado
    title = 'Valores perdidos de '+ param +' con respecto a la variable clase'
    x = aux.valperdidos
    color = clase
    barnorm = "fraction"
    a = px.histogram(x=x, color=color, barnorm=barnorm, title=title)
    return a
        

#Función para calcular el porcentaje de valores nulos en variables
def MissingValuesCount(Data, param):
    Nan= Data.isnull().sum()
    for s in param:
        Aux =Data[s]
        TotalVal= np.product(Aux.shape)
        NanSum = Aux.isnull().sum()
        result= (NanSum/TotalVal)*100  
        print(f"El porcentaje de valores nulos en la variable {s} es del {result:.2f}% ")


#Función para eliminar outliers
def outlier_rejection(X, y, seed):
    model = IsolationForest(random_state=seed)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]


#Función para realizar la evaluacion de bases de datos medicas
def EvaluationWClassRepo(model,
             X_train, X_test,
             y_train, y_test):
    
    clf = model.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    
    d = dict(enumerate(y_test.cat.categories))
    a="% s" % d.get(0)
    b="% s" % d.get(1)
    labels=[a,b]
    print(classification_report(y_test, y_pred, target_names=labels))
    
    disp = plot_confusion_matrix(clf, X_test, y_test)
    
    accuracy= accuracy*100

    disp.ax_.set_title(f" Tasa de precisión = {accuracy:.2f}"+"%")
    

# 2. Acceso y almacenamiento de datos

## *2.1 Breast cancer Winsconsin*

La base de datos Breast Cancer Winscosin es el resultado del análisis de una imagen digitalizada de un aspirado con aguja fina (FNA) de una masa mamaria. El análisis se realiza teniendo en cuenta las distinas variables que maneja la base de datos las cuales son:

1. ID number: Es un número creciente que servirá de identificador para cada uno de los casos de la base de datos.
2. Diagnosis: Variable categórica que guardará el resultado del análisis de la masa mamaria. Si tiene el valor B en caso de que la masa sea benignea y M en caso de que la masa sea maligna. Esta variable categórica será la que tomemos como variable objetivo.
3. radius: Variable continua real que guardará la media de las distancias desde el centro hasta los puntos del perímetro de la masa.
4. texture: Variable continua real que guardará la desviación estándar de los valores de la escala de grises de la masa.
5. perimeter: Variable continua real que guardará el perímetro de la masa.
6. area: Variable real que guardará el perímetro de la masa.
7. smoothness: Variable continua real que guardará la variación local en longitudes de radio de la masa.
8. compactness: Variable continua real resultante de la operación perimeter^2 / area - 1.0 (los valores de la variable real corresponde a los de la masa)
9. concavity: Variable continua real que representa la severidad de las porciones cóncavas del contorno de la masa.
10. concave points: Variable real que represente el numero de porciones cóncavas  del contorno de la masa.
11. symmetry: Variable continua real que representa la simetria de la masa.
12. fractal dimension ("coastline approximation" - 1): Variable continua real que representa la aproximación de la linea costera de la masa

Las dos primeras variables son variables de información y el resto son variables continuas que se computan para cada caso de la masa mamaria analizada. Para estas últimas variables habrá tres tipos para cada uno de los casos:

    -Variable_mean: Guardará el valor medio de la variable.
    -Variable_se: Guardará el error estándar de la variable.
    -Variable_worst: Guardará el peor valor (media de los tres valores más grandes) de la variable.

Como conclusion la base de datos se utilizará en nuestro estudio para generar un modelo que prediga según unos valores si la masa mamaria analiza es B (Benigna) o M (Maligna)


Cargamos la base de datos Breast cancer wisconsin  tratando la variable identificadora id como indice y la variable diagnosis como variable objetivo. Finalmente mostramos una muestra del conjunto de datos cargados

In [ ]:
filepathWisconsin = "../input/breast-cancer-wisconsin-data/data.csv"
indexWisconsin = "id"
targetWisconsin = "diagnosis"
dataWisconsin = utils.load_data(filepathWisconsin, indexWisconsin, targetWisconsin)
dataWisconsin.sample(5, random_state=seed)

Si realizamos una observación de esta muestra, podemos darnos cuenta de que destaca una variable llamada Unnamed 32, ya que en la muestra todos los sus valores son nulos (NaN). Si nos descargamos el fichero .csv de la base de datos Breast Cancer Wisconsin, podemos observar que esta variable es ruido ya que se ha introducido por error a la hora de cargar el conjunto de datos pues en el fichero se ha introducido una coma de más y al cargarlo esa coma la toma como otra variable, para ser exactos como la variable Unnamed32.
Por ello lo que vamos a hacer a continuación es eliminar dicha columna (mediante el método de pandas drop) del conjunto de datos dataWisconsin y enseñaremos otra muestra de dicho conjunto de datos para confirmar si se ha borrado correctamente.

In [ ]:
dataWisconsin = dataWisconsin.drop(dataWisconsin.columns[31], axis = 'columns')
dataWisconsin.sample(5, random_state=seed)

Tras confirmar que se ha borrado correctamente Unamed32, el siguiente paso consitirá en dividir el conjunto de datos en dos subconjuntos; uno con las variables predictoras (X) y otro con las variables objetivo (Y). Después mostramos una muestra (sample) de los subconjuntos creados.

In [ ]:
(XWisconsin, yWisconsin) = utils.divide_dataset(dataWisconsin, target="diagnosis")

In [ ]:
XWisconsin.sample(5, random_state=seed)

In [ ]:
yWisconsin.sample(5, random_state=seed)

A continuación vamos a dividir nuestros subconjunto de datos en otros dos:
*  uno que sirva como muestra de entrenemiento (XWisconsin_train y yWisconsin_train) (70% del subconjunto inicial)
*  el otro que sirva como muestra de prueba (XWisconsin_test y yWisconsin_test) (30% del subconjunto incial)

Esta división la realizaremos con el método train_test_split.

Finalmente mostraremos una muestra de cada uno de los subconjuntos obtenidos de forma aleatoria (utilizando la semilla que hemos definido al principio de la práctica).

In [ ]:
(XWisconsin_train, XWisconsin_test, yWisconsin_train, yWisconsin_test) = train_test_split(XWisconsin, yWisconsin,
                                                      stratify=yWisconsin,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
XWisconsin_train.sample(5, random_state=seed)

In [ ]:
XWisconsin_test.sample(5, random_state=seed)

In [ ]:
yWisconsin_train.sample(5, random_state=seed)

In [ ]:
yWisconsin_test.sample(5, random_state=seed)

## *2.2 Pima Indians diabetes*

La base de datos Pima Indians diabetes tiene el objetivo de predecir de forma diagnóstica si un paciente tiene o no diabetes, basándose en ciertas mediciones ya realizadas e incluidas en la base de datos. Algunas restricciones se han establecido en la seleccion de las instacias para la base de datos. En particular, todos los pacientes son mujeres con al menos 21 años con herencia del pueblo Pima (que és un grupo de indígenas de Estados Unidos que viven en Arizona). Las distitnas variables que maneja la base de datos son las siguientes:

1. Pregnancies: Variable discreta que indica el número de veces que ha estado embarazada la paciente.
2. Glucose: Variable continua entera que indica la concentración de glucosa en plasma de la paciente tras 2 horas de que se la haya realizado una prueba oral de tolerancia a la glucosa.
3. BloodPresure: Variable continua entera que indica la presión arterial diastólica de la paciente en mm/hg.
4. SkinThickness: Variable continua entera que indica el espesor del pliegue cutáneo del triceps de la paciente en mm
5. Insulin: Variable continua entera que indica el suelo insulino tras 2 horas de la paciente en mu U/ml
6. BMI: Variable continua real que indica el índice de masa corporal de la paciente dado por la divisón del peso en kg entre la altura en metros al cuadrado
7. DiabetesPedigreeFunction: Variable continua real que indica el resultado de la función de pedigree.
8. Age: Variable continua entera que indica la edad del paciente.
9. Outcome: Variable categórica que puede tener los valores 1 o 0. Esta variable categórica tomará el valor 1 en el caso de que la paciente tenga diabetes y tomará el valor 0 en el caso de que la paciente no tenga diabetes.

La variable objetivo de este problema sería outcome, mientras que el resto serán variables predictoras.
Como conclusión, recordar que los modelos generados por esta base de datos tendrán la finalidad de que, dependiendo de los valores de las variables predictoras, intentar diagnosticar si un paciente tiene diabetes o no "rellenando" el valor de la variable objetivo Outcome.



Lo primero que vamos a hacer es cargar la base de datos utilizando Outcome como variable objetivo y como la base de datos carece de una variable identificadora que se pueda utilizar como índice, no utilizaremos ninguna variable (None) como variable indice ya que en ese caso, el método load_data generá un indice de forma automática.

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"
indexDiabetes = None
targetDiabetes = "Outcome"
dataDiabetes = utils.load_data(filepath, indexDiabetes, targetDiabetes)
dataDiabetes.sample(5, random_state=seed)

En el siguiente paso dividimos el conjunto de datos en dos subconjuntos, uno con las variables predictoras (X) y otro con las variables objetivo (Y). Después mostramos una muestra (sample) de los subconjuntos creados.

In [ ]:
(XDiabetes, yDiabetes) = utils.divide_dataset(dataDiabetes, target="Outcome")

In [ ]:
XDiabetes.sample(5, random_state=seed)

In [ ]:
yDiabetes.sample(5, random_state=seed)

A continuación vamos a dividir nuestros subconjunto de datos en otros dos:
*  uno que sirva como muestra de entrenemiento (XDiabetes_train y yDiabetes_train) (70% del subconjunto inicial)
*  el otro que sirva como muestra de prueba (XDiabetes_test y yDiabetes_test) (30% del subconjunto incial)

Esta división la realizaremos con el método train_test_split.

Finalmente mostraremos una muestra de cada uno de los subconjuntos obtenidos de forma aleatoria (utilizando la semilla que hemos definido al principio de la práctica).

In [ ]:
(XDiabetes_train, XDiabetes_test, yDiabetes_train, yDiabetes_test) = train_test_split(XDiabetes, yDiabetes,
                                                      stratify=yDiabetes,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
XDiabetes_train.sample(5, random_state=seed)

In [ ]:
XDiabetes_test.sample(5, random_state=seed)

In [ ]:
yDiabetes_train.sample(5, random_state=seed)

In [ ]:
yDiabetes_test.sample(5, random_state=seed)

### **Preeliminares del analisis exploratiro**



Antes de empezar el analisis exploratorio obtendremos los conjuntos de entrenamiento y de test para todas las bases de datos sobre las que realizaremos el análisis exploratorio. Para ello volvemos a unir (usando el método join_dataset del fichero utils) los conjuntos de variables predictoras con la variable clase de los subconjuntos de entrenamiento y de test de todas las bases de datos.

In [ ]:
dataWisconsin_test= utils.join_dataset(XWisconsin_test,yWisconsin_test)
dataWisconsin_train= utils.join_dataset(XWisconsin_train,yWisconsin_train)

dataDiabetes_test= utils.join_dataset(XDiabetes_test,yDiabetes_test)
dataDiabetes_train= utils.join_dataset(XDiabetes_train,yDiabetes_train)

# 3. Analisis exploratorio

## *3.1 Breast cancer Winsconsin*

### **Descripción del conjunto**

Tendremos que tener conocimento de:
* Numeros de casos
* Tipos de variables 

Primero mostramos el número de casos y variables del conjunto de datos de entrenamiento con el metodo .shape

In [ ]:
dataWisconsin_train.shape

Podemos observar que el conjunto de datos tiene 398 casos y 31 variables,siendo estas quizas demasiadas para que se reprensenten con claridad en algunos de los gráficos que vamos a utilizar

In [ ]:
dataWisconsin_train.info(memory_usage=False)

A continuación podemos observar gracias al metodo .info ,que devuelve información del conjunto de datos incluyendo el tipo de sus variables, que de cada una de las 31 variables que hay 30 son del tipo real y una que es del tipo categorico (Que es la variable que usaremos como variable objetivo, diagnosis)

Continuaremos mostrando los distitnos valores que pueden tomar nuestras variables categóricas con el método .categories:

In [ ]:
yWisconsin_train.cat.categories

Podemos observar que diagnosis es una variable categórica que puede tomar el valor B en caso de que el resultado del diagnosis de la masa del paciente sea benigna y M en el caso de que el resultado del diagnosis de la masa del paciente sea maligno.

### **Visualización de las variables**

Vamos a empezar realizando un histograma sobre el conjunto de datos de entrenamiento:

In [ ]:
utils.plot_histogram(dataWisconsin_train)

Tras observar el histograma que hemos obtenido sobre el conjunto de datos de entrenamiento, podemos sacar las siguientes conclusiones:
* La primera conclusión que podemos sacar esque cada una de las variables perteneciente a un grupo (mean, se o worst) poseen una distribución similar, con alguna excepción. Por ejemplo, las variables del grupo del valor medio (mean), tienen una distribución normal con tendencia central en forma de campana de gauss, a excepción, de las variables continuas area_mean, compactness_mean, concavity_mean y concave points_mean, que tienen una distribución con tendencia exponencial decreciente. Por otra parte, las variables del grupo del error medio (se) poseen una distribución exponencial decreciente apreciable en todas las variables del grupo. Finalmente, las variables del grupo del peor valor (worst) tienen una distribución normal con tenencia central en forma de campana de gauss, a excepción, de las variables continuas area_worst compactness_worst, concavity_worts y fractal dimension_worst.
* La segunda conclusión que podemos sacar esque entre las 31 variables del conjunto de datos de entrenamiento, no hemos observado que haya presencia de datos no válidos ni indicios de outliers. No obstante; durante la realización de este análisis exploratorio realizaremos un diagrama de caja para cada una de las variables para analizar de forma más detallada la presencia de outliers.

A continuacion vamos a realizar un diagrama de barras:

In [ ]:
utils.plot_barplot(dataWisconsin_train)
B,M= yWisconsin_train.value_counts()
print('Numero de masas benigneas: ',B)
print('Numero de masas malignas : ',M)

Observamos que en nuestro conjunto de datos entrenamientos hay más casos en el que el diagnóstico final fue benigneo(250)que en el que el diagnóstico final fue maligno (148), esto quiere decir que el problema está desbalanceado.

Continuaremos con el análisis exploratorio realizando un diagrama de puntos:

In [ ]:
utils.plot_pairplot(dataWisconsin_train, target="diagnosis")

Lo único que podemos observar es lo que hemos mencionado al principio de este análisis, al ser 30 variables predictoras, algunas representaciones gráficas pueden no verse con claridad y este es un ejemplo. Por ello lo que vamos a hacer es dividir el diagrama de puntos entre tres, uno para cada uno de los tres tipos que pueden tener las variables predictoras: mean, se y worst.

In [ ]:
mean_train = dataWisconsin_train.iloc[:,[0,1,2,3,4,5,6,7,8,9,30]]
se_train = dataWisconsin_train.iloc[:,[10,11,12,13,14,15,16,17,18,19,30]]
worst_train = dataWisconsin_train.iloc[:,[20,21,22,23,24,25,26,27,28,29,30]]
utils.plot_pairplot(mean_train, target="diagnosis")

In [ ]:
utils.plot_pairplot(se_train, target="diagnosis")

In [ ]:
utils.plot_pairplot(worst_train, target="diagnosis")

Respecto a estas graficas podemas darnos cuenta que las variables continuas radius, perimeter y area estan muy relacionados (en los tres grupos de variables predictoras mean, se y worst), obviamente esto puede ser por que para obtener el perimetro y el area es necesario saber el radio. Entonces, sería factible descartar radius ya que sería una variable simple de la que se obtienen las variables derivadas perimeter y area, y estas dos tendrías más información a la hora de generar los modelos que la variable radius.
También podemos observar que la mejor forma de realizar una discretización óptima sería utilizando dos intervalos y quizas la mejor estrategia sería utilizar KMeans, pero hay tantos casos que no se puede estar seguro a simple vista.

Para comprobar algunas de las conclusiones que hemos realizado en el apartado anterior vamos a generar una matriz de correlación:

Al igual que para el gráfico anterior vamos a realizar tres matrices distintas una para cada uno de los tipos que pueden tener las variables predictoras; mean,se y worst.

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(mean_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(se_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(worst_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Las conclusiones que podemos sacar esque en los tres grupos de variables predictoras podemos presenciar que hay dos ternas de variables predictoras donde se presencia un alto grado de correlación entre sus miembros:
* La primera terna consta de las variables continuas radius, perimeter y área, las cuales presentan un alto nivel de correlación debido a que perimeter y área son variables derivadas de radio. 
* La segunda terna consta de las variables continuas concavity, concave points y compactness las cuales también presentan un alto nivel de correlación debido a que compactness y concavity son variables derivadas de concave points.
Por lo tanto, en el preprocesamiento no deberiamos de tener en cuenta las variables simples(radius y concave points) y utilizar solo las variables derivadas de estas ultimas ya que aportarán más información al modelo.


Diagrama de caja

In [ ]:
plot_boxplot(dataWisconsin_train)

Debido a que estamos utilziando el método .iplot para generar los diagramas de cajas y que este método no genera diagramas de caja en los que se resalten los outliers presentes en una variable (como si hacen otros diagramas de cajas generados mediante seaborn o plotly express), y que no sabemos si estos gráficos serán visibles cuando hagamos el commit del notebook; hemos decidido apoyar las conclusiones que sacariamos de un diagrama de cajas mediante una función que hemos creado nosostros mismos. 
Dicha función se llama calc_outliers y tras pasarle un conjunto de datos y el nombre de las variables de dicho conjunto de datos en las que queremos buscar la presencia de outliers como parámetros, te devuelve el número de outliers presentes en cada una de las variables solicitadas, un array con cada uno de los outliers presentes en cada variable y una conclusión final con el número total de outliers en el conjunto de datos, la media de outliers por variable en el conjunto de datos, la variable con el mayor número de outliers con el número de outliers que posee y la variable con el menor número de outliers con el corresponideinte número de outliers.
Dicho esto, el resultado de la ejecución de dicha función para el conjunto de datos de entrenamiento de Winsconsin pasándole tambíen como parámetro el nombre de todas las variables predictoras del conjunto sería el siguiente:

In [ ]:
calc_outliers(dataWisconsin_train, ['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean',
      'radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se',
      'radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst',]
)

Por lo tanto, las conclusiones que podemos sacar esque, aunque hay una variable predictora en el conjunto de datos sin ningún outlier (concave points_worst) el número total de outliers que tenemos en el conjunto de datos es demasiado grande (435) y dichos outliers están presentes en prácticamente todas las variables predictoras, por lo que se probará a utilizar un estimador que elimine los outliers para comprobar si mejoran los resultados de los distintos modelos al usarlo.

## *3.2 Pima Indians diabetes*

### **Descripción del conjunto**

Tendremos que tener conocimento de:
* Numeros de casos
* Tipos de variables 

Primero mostramos el número de casos y variables del conjunto de datos de entrenamiento con el metodo .shape

In [ ]:
dataDiabetes_train.shape

El número de casos del conjunto de datos es 537 mientras que el número de variables es 9

A continuación mostramos el tipo de cada una de las variables del conjunto de datos con el método .info:

In [ ]:
dataDiabetes_train.info(memory_usage=False)

Donde podemos observar que de las 9 variables, todas son de tipo entero excepto tres; BMI y DiabetesPedigreeFunction que son de tipo real (dato que tendremos que tener en cuenta si las modificamos en el preprocesamiento), y Outcome que es de tipo categórico y es la que tomaremos como variable objetivo.

A continuación mostramos los distitnos valores que pueden tomar nuestras variables categóricas con el método .categories:

In [ ]:
yDiabetes.cat.categories

Podemos observar que la variable Outcome es una variable categórica numérica que puede tomar los valores 0 o 1 en caso negativo o afirmativo de que el paciente tenga diabetes respectivamente.

### **Visualización de las variables**

Vamos a empezar realizando un histograma sobre el conjunto de datos de entrenamiento:

In [ ]:
utils.plot_histogram(dataDiabetes_train)

Si observamos la gráficas podemos observar dos detalles importantes:
* Lo primero es que no todas las variables predictoras tienen una distribución normal con tendencia central en forma de campana de gauss, solamente las variables predictoras SkinThickness, BMI, BloodPresure y Glucose; mientras que el resto de variables predictoras ( Age, Pregnancies, Insulin y DiabetesPedigreeFunction) poseen una distribución con tendencia exponencial decreciente. De hecho en ciertas variables predictoras como DiabetesPedigreeFunction o Insulin, en sus gráficas podemos empezar a apreciar la aparición de outliers que comprobaremos más tarde con los gráficos de caja.
* Lo segundo es que en las gráficas se puede observar como ciertas variables continuas toman valores perdidos, es decir toman el valor 0 cuando según la lógica de los valores que pueden tomar dichas variables continuas, sería imposible que dichas variables continuas tuvisen como valor un 0. Dichas variables continuas son Glucose (un paciente no puede tener 0 de glucosa), BloodPresure (un paciente no puede tener 0 de presión sanguínea), SkinThickness (la piel de un paciente debe de tener grosor), Insulin (un paciente ha de tener insulina) y BMI (el índice de masa corporal de una persona no puede ser 0).

A continuación debido a las conclusiones del histograma, vamos a obtener el porcentaje de ceros que tienen estas variables continuas con el método del fichero de utilidad ZeroCount y así comproboremos la cantidad de ruido que tienen estas variables continuas y si vale la pena tenerlas en cuenta para el preprocesamiento:

In [ ]:
param=["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

utils.ZeroCount(dataDiabetes_train,param)

Lo que podemos observar esque el porcentaje de ruido que tienen variables continuas como BloodPresure, Glucose o BMI es aceptable, mientras que el de las variables continuas Insulin y SkinThickness (48.60% y 29.24% de ruido respectivamente) es demasiado alto, por lo tanto, a continuación vamos a analizar la potencia discriminativa de los valores perdidos de estas variables continuas con respecto a la variable clase, antes de concluir en si se deberían o no tenerse en cuenta a la hora de generar los modelos.

Para ello vamos a recurrir al método _ValPerdidos_VarClase de nuestro fichero de utilidad, al cuál tras pasarle como parámetros un conjunto de datos, el nombre de una variable predictora y la variable objetivo del conjunto de datos; creará un diagrama de barras "apilado" con dos barras, siendo la primera una representacion normalizada de los casos del conjunto de datos en los que la variable predictora (que hemos pasado como parámetro)  no tiene valores perdidos y la segunda la que si tiene valores perdidos. Además, cada una de las barras estará dividida en base a los valores de la variable objetivo que hemos pasado como parámetro anteriormente, representado el porcentaje de casos del conjunto de datos que toman los distintos valores de dicha variable objetivo(en el caso de la variable objetivo del conjunto de datos dataDiabetes_train dichos valores serán 0 o 1). Este tipo de gráfico nos pérmitirá comprobar la potencia discriminativa de los valores perdidos de una variable predictora con respecto a la variable clase.

Explicado el método, lo vamos a ejecutar para la variable predictora SkinThickness cuyos valores son en un 29,6% valores perdidos:

In [ ]:
utils._ValPerdidos_VarClase(dataDiabetes_train,"SkinThickness",dataDiabetes_train.Outcome)

Y después lo vamos a ejecutar para la variable predictora Insulin cuyos valores son en un 48,60% valores perdidos:

In [ ]:
utils._ValPerdidos_VarClase(dataDiabetes_train,"Insulin",dataDiabetes_train.Outcome)

Como podemos apreciar, los valores perdidos de estas dos variables predictoras no tienen una gran potencia discriminatoria, ya que la diferencia de porcentajes de casos en base al valor de la variable predictora Outcome no varía mucho entre los casos con valores perdidos y sin valores perdidos:
* ± 0.05 (±5%) en el caso de la variable predictora SkinThickness.
* ± 0.03 (±3%)en el caso de la variable predictora Insulin.

Por lo tanto podemos concluir, que al carecer de poder discriminativo los valores perdidos de las variables SkinThickness e Insulin y al poseer ambas un alto porcentaje de valores perdidos (29,6% y 48,60%), es recomendable que no las tengamos en cuenta a la hora de generar nuestros modelos.

A continuacion vamos a realizar un diagrama de barras:

In [ ]:
utils.plot_barplot(dataDiabetes_train)
S,N= yDiabetes_train.value_counts()
print('Numero de Diabetes: ',S)
print('Numero de no Diabetes: ',N)

Podemos observar que en nuestro conjunto de datos hay más casos en los que la variable objetivo Outcome indicaba que el paciente tenia diabetes (350) que en los que la variable objetivo Outcome indicaba que el paciente no tenia diabetes (187). Al haber tanta diferencia entre el número de los casos en los que el paciente tiene o no diabetes podemos decir que el problema está desbalanceado.

Continuaremos con el análisis exploratorio realizando un diagrama de puntos:

In [ ]:
utils.plot_pairplot(dataDiabetes_train, target="Outcome")

Podemos observar que hay ciertas variables predictoras muy relacionadas entre si, como Age y Pregnancies, Glucose y Age o Glucose e Insulin; por lo que quizas en procesos posteriores deberiamos eliminar alguno de los miembros de estos pares de variables predictoras. Tambíen podemos darnos cuenta de que si realizamos una discretización quizas deberíamos de realizarla con 3 intervalos, el problema, esque los datos están tan juntos que en principio no podemos definir ninguna estrategia de discretización correcta para este conjunto de datos.

Para comprobar algunas de las conclusiones que hemos realizado en el apartado anterior vamos a generar una matriz de correlación:

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(dataDiabetes_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Al observar la matriz nos podemos dar cuenta de que el par de variables predictoras Glucose-Age y Glucose-Insulin están relacionados pero no lo suficiente para eliminar una de estas variables en el preprocesamiento; no obstante, el par de variables Age-Pregnancies es el más relacionado del conjunto de datos y quizas deberiamos de eliminar una de estas variables predictoras durante el preprocesamiento, siendo quizas, la más favorable a esta eliminación la variable predictora pregnancies, ya que en principio podemos suponer que aporta menos información que la variable predictora Age a los modelos que vamos a generar para nuestro conjunto de datos pues creemos que la edad(Age) es un factor más relevante para la aparición de la enfermedad diabetes que el número de embarazos(Pregnancies).

Para acabar el análisis exploratorio vamos a realizar un diagrama de cajas para comprobar si podrían haber outliers en las variables del conjunto de datos:

In [ ]:
utils.plot_boxplot(dataDiabetes_train)

Aplicando la misma justificación que hemos desarrolado para los diagramas de cajas del conjunto de datos de Winsconsin, vamos a ejecutar la función calc_outliers para apoyar los resultados del diagrama de cajas:

In [ ]:
calc_outliers(dataDiabetes_train, ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
)

Las conclusiones que podemos sacar esque el total de valores outliers en el conjunto de datos es bastante alta (114) y estan presentes en todas las variables predictoras del conjunto de datos, siendo la variable predictora que menos tiene 1, pero sigue teniendo como mínimo un outlier. Por lo tanto se justifica el uso de un estimador cuya función sea eliminar outliers para modelar este conjunto de datos y así comprobar si mejoran los resultados con respecto a un modelo que no elimine dichos outliers.

# 4. Preprocesamiento de datos

En esta parte aplicaremos el preprocesamiento en base a las modificaciones que hemos visto que necesitan los conjuntos de datos durante el análisis exploratorio.

Empezamos el preprocesamiento definiendo el estimador que hara uso de la función outlier_rejection del fichero de utilidad utils para eliminar outliers. Este estimador se utilizará posteriormente en la generación de modelos aplicandosé al conjunto de datos.

In [ ]:
#Estimador para eliminar outliers
OutlierRejection = FunctionSampler(func=utils.outlier_rejection, kw_args={"seed":seed})

## *4.1 Breast cancer Winsconsin*

Como hemos concluido en el análisis exploratorio, en los tres grupos de variables predictoras (mean,se y worst) se pueden apreciar dos ternas de variables predictoras, donde estas están muy correlacionadas entre sí, ya que en cada una de estas ternas hay dos variables derivadas de la tercera variable de la terna que es una variable simple. Por ello en el preprocesamiento vamos a generar un estimador que elimine las dos variables simples (radius y concave points) de todos los grupos de variables (mean,se y worst) del conjunto  de datos para que no influyan en la generación del modelo ya que aportan menos información de la que aportan las variables que derivan de ellos.
Para la generación del estimador vamos a utilizar el método make_column_transformer de la libreria sklearn.compose y vamos a indicar como parámetro del transformador 'drop' para que elimine la lista de variables predictoras de las que hemos hablado con anteriorioridad, además; hemos indicado que para el resto de variables del conjunto el estimador simplemente no las toque igualando el parámetro remainder como 'passthrough'.

In [ ]:
preproceserWinsconsin = make_column_transformer(('drop',['radius_mean','radius_se','radius_worst','concave points_mean',
          'concave points_se','concave points_worst']),
                                                remainder='passthrough')


Como dato aclaratorio, también podriamos haber generado el estimador utilizando el transformador 'passthrough' y seleccionado todas las variables predictoras con las que nos queremos quedar sin igualar el parámetro remainder a passthrough, pues esto hará que las variables que no le especifiquemos las borrará del conjunto de datos; básicamente es hacer el proceso que hemos hecho con el estimador al revés.

## *4.2 Pima Indians diabetes*

Como hemos indicado en el preprocesamiento de esta base de datos lo que tendremos que hacer será:
* Sustituir los valores cero de las variables continuas Glucose, BloodPressure y BMI
* Eliminar una variable predictora del par de variables predictoras Age y Pregnancies por su alto coeficiente de correlación
* Eliminar las variables predictoras Insulin y SkinThickness por la alta cantidad de valores perdidos que tienen y la ausencia de poder discriminativo de estos.
* Dejar el resto de variables del conjunto igual.

Para la sustitución de valores cero de las variables predictoras Glucose, BloodPressure y BMI utilizaremos el estimador SimpleImputer pero teniendo en cuenta que BMI se trata de una variable real y que las otras dos se tratan de variables enteras, tendremos que generar un estimador para cada tipo de variable (entera y real) modificando la estrategia de búsqueda, pues aunque lo normal es utilizar la media para la sustitución de valores ruidosos, al ser Glucose y BloodPresure enteros, no se puede introducir de repente un valor con decimales (la media podría tener o no decimales) por ello el simpleImputer que afectará a las variables enteras utilizará la estrategia "median" sustituyendo los ceros por el valor más "centrado" entre todos  y que claramente será de tipo entero y sin decimales. Ambos SimpleImputer (de enteros y decimales) los "combinaremos" con un método make_column_transformer donde le pasaremos a cada estimador las variables correspondientes asignándolas al parámetro pattern del make_column_selector. Para dejar al resto de variables igual simplemente declararemos las variables predictoras que no se deben de tocar en un parámetro aparte y se introducirán en el make_column_transformer pero en vez de pasarle un estimador como parámetro introduciremos el valor 'passthrough' que simplemente dejará igual las variables predictoras. Finalmente si queremos eliminar las variables predictoras que hemos acordado en el análisis exploratorio bastará con no incluirlas en el make_colum_transformer.

In [ ]:
#Variables para sustituir ceros
features1 = 'Glucose|BloodPressure'
features2 = 'BMI'
#Variables que no hay que tocar
features3 = 'DiabetesPedigreeFunction|Age'

#Estimador para integers
replace0Integer_Estimator = make_pipeline(SimpleImputer(strategy="median",missing_values=0 ))
#Estimador para reales
replace0Float_Estimator = make_pipeline(SimpleImputer(strategy="mean",missing_values=0 ))

preproceserDiabetes = make_column_transformer(
    (replace0Integer_Estimator, make_column_selector(pattern= features1)),
    (replace0Float_Estimator, make_column_selector(pattern= features2)),
     ('passthrough', make_column_selector(pattern= features3)))



# 5. Algoritmos de clasificación

En esta parte generaremos los modelos que vamos a evaluar y las pipelines que saldrán como resultado de aplicar a dichos modelos los estimadores que hemos creado en el apartado anterior (que también evaluaremos posteriormente).

Los modelos que evaluaremos serán el modelo zero_r y el modelo de arbol de decision que usará la semilla que estamos usando durante todo el problema.

In [ ]:
#Generación del modelo zero_r
zero_r_model = DummyClassifier(strategy="most_frequent")
#Generación del modelo de árbol de decision
tree_model = DecisionTreeClassifier(random_state=seed)

## *5.1 Breast cancer Winsconsin*

Para empezar generamos los distintos discretizadores que aplicaremos al modelo de árbol de decisión. Como hemos mencionado en el análisis exploratorio utilizaremos 2 intervalos y probaremos todas las estrategias al no tener claro del todo cual hay que utilizar.

In [ ]:
discretizerUWinsconsin = KBinsDiscretizer(n_bins=2, strategy="uniform")
discretizerQWinsconsin = KBinsDiscretizer(n_bins=2, strategy="quantile")
discretizerKWinsconsin = KBinsDiscretizer(n_bins=2, strategy="kmeans")


A continuación le aplicaremos al modelo de árbol de decisión el estimador con el preprocesamiento que hemos desarrolado en el apartado anterior. No se lo aplicamos al modelo zeroR porque no le afectaria. Tambien le aplicamos el estimador al árbol de decisión discretizado con 2 intervalos y todas las posibles estrategias de discretización.

In [ ]:

#Modelo de árbol de decisón con eliminación de variables predictoras muy relacionadas
preprocess_treemodel_Winsconsin=make_pipeline(preproceserWinsconsin, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones y con eliminación
# de variable predictoras muy relacionadas
preprocess_treemodel_discretizeU_Winsconsin=make_pipeline(preproceserWinsconsin,discretizerUWinsconsin,tree_model)
preprocess_treemodel_discretizeQ_Winsconsin=make_pipeline(preproceserWinsconsin,discretizerQWinsconsin,tree_model)
preprocess_treemodel_discretizeK_Winsconsin=make_pipeline(preproceserWinsconsin,discretizerKWinsconsin,tree_model)

Finalmente hemos generado cuatro pipelines extra, que son las mismas cuatro pipelines anteriores pero añadiéndoles un estimador que elimina outliers. Este estimador lo hemos añadido en cuatro pipelines extra aparte en vez de integrarlo todo junto porque sabemos que el proceso de eliminación de outliers puede generar sobreajuste y empeorar los resultados de la evaluación; por lo tanto vamos a evaluar tanto las pipelines sin eliminación de outliers como las que si que lo hacen y comparar los resultados obtenidos en el siguiente apartado.

In [ ]:

#Modelo de árbol de decisón con eliminación de variables predictoras muy relacionadas 
#eliminación de outliers
extra_preprocess_treemodel_Winsconsin=make_pipeline(OutlierRejection, preproceserWinsconsin, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones y con eliminación
# de variable predictoras muy relacionadas y eliminación de outliers
extra_preprocess_treemodel_discretizeU_Winsconsin=make_pipeline(preproceserWinsconsin,OutlierRejection,discretizerUWinsconsin,tree_model)
extra_preprocess_treemodel_discretizeQ_Winsconsin=make_pipeline(preproceserWinsconsin,OutlierRejection,discretizerQWinsconsin,tree_model)
extra_preprocess_treemodel_discretizeK_Winsconsin=make_pipeline(preproceserWinsconsin,OutlierRejection,discretizerKWinsconsin,tree_model)



## *5.2 Pima Indians diabetes*

Para empezar generamos los distintos discretizadores que aplicaremos al modelo de árbol de decisión. Como hemos mencionado en el análisis exploratorio utilizaremos 3 intervalos y probaremos todas las estrategias al no tener claro del todo cual hay que utilizar

In [ ]:
discretizerUDiabetes = KBinsDiscretizer(n_bins=3, strategy="uniform")
discretizerQDiabetes = KBinsDiscretizer(n_bins=3, strategy="quantile")
discretizerKDiabetes = KBinsDiscretizer(n_bins=3, strategy="kmeans")

A continuación le aplicaremos al modelo de árbol de decisión el estimador con el preprocesamiento que hemos desarrolado en el apartado anterior. No se lo aplicamos al modelo zeroR porque no le afectaria. Tambien le aplicamos el estimador al árbol de decisión discretizado con 3 intervalos y todas las posibles estrategias de discretización.

In [ ]:
#Modelo de árbol de decisión con sustitución de ceros
preprocess_treemodel_Diabetes=make_pipeline(preproceserDiabetes, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones y sustitución de ceros
preprocess_treemodel_discretizeU_Diabetes=make_pipeline(preproceserDiabetes,discretizerUDiabetes,tree_model)
preprocess_treemodel_discretizeQ_Diabetes=make_pipeline(preproceserDiabetes,discretizerQDiabetes,tree_model)
preprocess_treemodel_discretizeK_Diabetes=make_pipeline(preproceserDiabetes,discretizerKDiabetes,tree_model)


Finalmente hemos generado cuatro pipelines extra, que son las mismas cuatro pipelines anteriores pero añadiéndoles un estimador que elimina outliers. Este estimador lo hemos añadido en cuatro pipelines extra aparte en vez de integrarlo todo junto porque sabemos que el proceso de eliminación de outliers puede generar sobreajuste y empeorar los resultados de la evaluación; por lo tanto vamos a evaluar tanto las pipelines sin eliminación de outliers como las que si que lo hacen y comparar los resultados obtenidos en el siguiente apartado.

In [ ]:

#Modelo de árbol de decisión con sustitución de ceros y eliminación de outliers
extra_preprocess_treemodel_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones, sustitución de ceros y eliminación de outliers
extra_preprocess_treemodel_discretizeU_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection,discretizerUDiabetes,tree_model)
extra_preprocess_treemodel_discretizeQ_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection,discretizerQDiabetes,tree_model)
extra_preprocess_treemodel_discretizeK_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection,discretizerKDiabetes,tree_model)

# 6. Evaluacion de modelos

El proceso de evaluación de modelos lo realizaremos siguiendo los siguientes pasos para cada uno de los subapartados de la evaluación (uno para cada uno de las bases de datos que hemos tratado):
1. Mencion de la métrica que utilizaremos para elegir el mejor modelo de los evaluados, que dependerá de la naturaleza del problema del que trata la base de datos que utilicemos
2. Mostrar el resultado de la evaluación sobre el conjunto de datos correspondiente de los modelos que usan los algoritmos ZeroR y el árbol de decisión sin inclusión de estimadores extra, el resultado de la evaluación sobre el conjunto de datos correspondiente de la pipeline resultado de aplicarle al árbol de decisión el preprocesamiento básico del conjunto de datos correspondiente y otras tres pipelines que le aplican al árbol de decisón las tres tipos de discretizaciones mencionadas anteriormente y el preprocesamiento básico; y finalmente el resultado de la evaluación sobre el conjunto de datos corresponidente de las últimas 4 pipelines añadiéndoles un estimador de eliminación de outliers. La evalución se realizará utilizando la función del fichero de utilidad utils EvaluationWClassRepo
3. Redacción de una pequeña conclusión que resuma los resultados obtenidos y los detalles más importantes de estos.

## *6.1 Breast cancer Winsconsin*

Teniendo en cuenta que la base de datos Breast cancer winsconsin surge de un problema médico y por ello, la métrica normalmente utilizada para estos problemas es el recall o tasa de verdaderos positivos, es la que tendremos que que emplear para la evaluación. Dicha tasa la podemos ver en la estructura classsification report que generará el método EvalutationWClassRepo. 
Finalmente, al utilizarse la base de datos para predecir el resultado del análisis de una masa mamaria, consideraremos que dicho análisis da positivo si el diagnóstico es maligno,es decir; la tasa de veraderos positivos se observará teniendo en cuenta que un positivo quiere decir que la masa mamaria es maligna.

> ### *Modelos sin preprocesamiento*

In [ ]:
utils.EvaluationWClassRepo(zero_r_model,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(tree_model,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

> ### *Modelos con el preprocesamiento desarrollado anteriormente*

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeU_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeQ_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeK_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)


> ### *Modelos con el preprocesamiento extra*

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeU_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeQ_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeK_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

> ### *Conclusiones*

Las conclusiones que hemos obtenido tras analizar los resultados de la evaluación de modelos son las siguientes:
* Los dos mejores modelos son el modelo del árbol de decisión con el preprocesamiento básico y con discretización de igual frecuencia y el modelo del árbol de decisión con el preprocesamiento extra y con una discretización de igual frecuencia. Ambos tienen una tasa de verdaderos positivos del 0.92.
* Si tenemos en cuenta la tasa de verdaderos positivos media (La media de la tasa de verdaderos positivos para positivo benigneo y positivo maligno), el mejor modelo sería el árbol de decisión con el preprocesamiento extra y con una discretización de igual frecuencia, con un tasa de verdaderos positivos media del 0.93.
* En general hemos obtenido una tasa de verdaderos positivos alta para todos nuestros modelos (el más bajo sin contar el modelo ZeroR ha sido 0.64, que pertenece al modelo del árbol de decisión con el preprocesamiento básico y con una discretización de igual anchura).
* El preprocesamiento extra,la eliminación de outliers, es posible que haya sobreajustado la mayoria de modelos generados para este conjunto de datos con este preprocesamiento, ya que los modelos que han eliminado outliers han tenido peores resultados que su contraparte sin eliminación de outliers(o iguales en el caso del modelo del árbol de decisión con preprocesamiento extra y con una discretización de igual frecuencia).
* La mejor discretización para el modelo de árboles de decisión es la de igual frecuencia, eliminado o no outliers, teniendo ambos una tasa de verdaderos positivos del 0.92.

## *6.2 Pima Indians diabetes*

Teniendo en cuenta que la base de datos Pima Indians diabetes surge de un problema médico y por ello, la métrica normalmente utilizada para estos problemas es la que tendremos que emplear para la evaluación, el recall o tasa de verdaderos positivos. Dicha tasa la podemos ver en la estructura classsification report que generará el método EvalutationWClassRepo. 
Finalmente, al utilizarse la base de datos para predecir de forma diagnóstica si un paciente tiene diabete o no, consideraremos que dicho análisis da positivo si la variable objetivo Outcome tiene el valor 1, es decir; la tasa de verdaderos positivos se observará teniendo en cuenta que un positivo quiere decir que el paciente tiene diabetes.

> ### *Modelos sin preprocesamiento*

In [ ]:
utils.EvaluationWClassRepo(zero_r_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(tree_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

> ### *Modelos con el preprocesamiento desarrollado anteriormente*

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeU_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeQ_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeK_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

> ### *Modelos con el preprocesamiento extra*

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeU_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeQ_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeK_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

> ### *Conclusiones*

Las conclusiones que hemos obtenido tras analizar los resultados de la evaluación de modelos son las siguientes:
* El mejor modelo que hemos obtenido ha sido el modelo del árbol de decisión sin discretización y con el preprocesamiento básico con una tasa de verdaderos positivos de 0.62.
* Si tenemos en cuenta la tasa de verdaderos positivos media (La media de la tasa de verdaderos positivos para positivo de diabetes y positivo de que no tenga diabetes), el mejor modelo es el del árbol de decisión con el preprocesamiento básico y con una discretización basada en igual anchura, pues su tasa de verdaderos positivos media es de 0,71.
* En general la tasa de veraderos positivos que hemos obtenidos de todos los modelos es bastante baja, siendo la más alta 0.62 y la más baja (sin tener en cuenta la del modelo ZeroR) ha sido 0,44; siendo esta a la vez la tasa de verdaderos positivos del modelo del árbol de decisión con el preprocesamiento básico y con una discretización basada en k-medias y la del modelo del árbol de decisión con el preprocesamiento extra y con una discretización basada en igual anchura.
* La eliminación de outliers ha podido sobreajustar parte de los modelos que usaban el preprocesamiento extra, ya que en la mayoría de estos modelos se han obtenido peores resultados para la tasa de verdaderos positivos con respecto al mismo modelo con el preprocesamiento básico, siendo una excepción el modelo  de árbol de decisión con discretización basado en K medians cuya tasas de verdaderos positivos es 0.46 cuando se eliminan outliers y 0.44 sin la eliminación de outliers.
* En el caso de la discretización, lo más recomendable es no realizar una discretización a este conjunto de datos si usamos como modelo un árbol de decisión, sin embargo la discretización que mejores resultados ha dado sin eliminación de outliers es la de igual frecuencia (con una tasa de verdaderos positivos de 0.59) y con eliminación de outliers es la basada en K medians (con una tasa de verdaderos positivos de 0.46). No obstante; están ambas por debajo del modelo del árbol de decisión sin discretizar con o sin eliminación de outliers (pero como mínimo con el preprocesamiento básico).
